In [24]:
#Importing all the modules
import pandas
import os

In [25]:
#First Step
#Making a function that will find the gap elements of a single file an return the data
def find_gap_ranges(data):
    gap_ranges = []
    prev_chr = None
    prev_end = None

    for element in data:
        chr, positions = element.split(":")
        chr = chr.replace(">","")
        start, end = map(int, positions.split("-"))

        if prev_chr is None or chr != prev_chr:
            prev_chr = chr
            prev_end = end
        else:
            if start-1 - prev_end+1 > 0:
                gap_ranges.append((chr, prev_end+1, start-1))
            prev_end = end

    return gap_ranges

In [26]:
#Making bed files for inaccessible areas
#Make sure you have run ./bedtools.sh inside /Rawfile/Access directory before you run this code
txt_dir = os.path.join("..","Bedtools","Access")
output_dir = os.path.join("..","Rawfile","NonAccess")
txt_files = [f for f in os.listdir(txt_dir) if f.endswith('.txt')]
print(txt_files)
for txt_file in txt_files:
    txt_path = os.path.join(txt_dir,txt_file)
    with open(txt_path, 'r') as file:
        lines = file.readlines()
    odd_position = lines[::2]
    gap_ranges = find_gap_ranges(odd_position)
    output_file = txt_file.replace(".txt","_in.bed")
    output_path = os.path.join(output_dir,output_file)
    with open(output_path, "w") as f:
        for chr, start, end in gap_ranges:
            f.write(f"{chr}\t{start}\t{end}\n")
  

['ENCFF825KFE.txt', 'ENCFF768XXW.txt', 'ENCFF986UZO.txt', 'ENCFF968PCT.txt', 'ENCFF928NLF.txt', 'ENCFF615ZGF.txt', 'ENCFF260PVK.txt', 'ENCFF621DXQ.txt', 'ENCFF139HDN.txt', 'ENCFF856GNF.txt']


In [27]:
#Cleaning bed files and tranforming into csv files by comparing both accessible and inaccessible areas
#Before you run this line, make sure you have run ./bedtools.sh in /Rawfile/NonAccess
access_dir = txt_dir
non_access_dir = os.path.join("..","Bedtools","NonAccess")
access_files = [f for f in os.listdir(txt_dir) if f.endswith('.txt')]

for txt_file in access_files:
    lines = []
    access_lines=[]
    non_access_lines = []
    access_path = os.path.join(access_dir,txt_file)
    
    non_access_file = txt_file.replace('.txt','_in.txt')
    non_access_path = os.path.join(non_access_dir,non_access_file)
    #Reads from access file
    with open(access_path,"r") as f:
        #lines += f.readlines()
        access_lines += f.readlines() #separate variable for access files
    #reads from non access file
    with open(non_access_path,"r") as f1:
        #lines += f1.readlines()
        non_access_lines += f1.readlines() #separate variable for nonaccess files
    #finding out the ATGCs of all the combined files
    
    lines = access_lines + non_access_lines
    even_lines = lines[1::2]

    #Average from both files floor division
    total_chars = sum(len(s) for s in even_lines)
    total_elements = len(even_lines)
    print(non_access_file)
    average = total_chars//total_elements
    
    print(f"Average: {average} Average from both access and non access region is too high \n")
    
    #So we only account to take the average from the accessible files 
    access_lines_even = access_lines[1::2]
    total_chars = sum(len(s) for s in access_lines_even)
    total_elements = len(access_lines_even)
    average = total_chars//total_elements
    print(f"After accounting the average for only accessible element, the average is: {average}\n")
    
    #Cleanign data for accessed element. Only extracting ATGCs from access and non access files

    nonaccess_lines_even = non_access_lines[1::2]
    
    access_chrom_data = []
    nonaccess_chrom_data = []
    #Extracting average or more than average ATGcs and trimming if the value is higher than the average
    
    for line in access_lines_even:
        if(average <= len(line)):
            main_data = line[:average].lower()
        
            access_chrom_data.append(main_data)


    #Doing same for non access files
    for line in nonaccess_lines_even:
        if(average <=len(line)):
            main_data = line[:average].lower()
            nonaccess_chrom_data.append(main_data)

    #Creating output files and directories
    access_output_file = txt_file.replace(".txt","_output.csv")
    nonaccess_output_file = non_access_file.replace(".txt","_output.csv")
    
    access_output_dir = os.path.join("..","Output","Access")
    nonaccess_output_dir = os.path.join("..","Output","NonAccess")
    
    access_output_path = os.path.join(access_output_dir,access_output_file)
    nonaccess_output_path = os.path.join(nonaccess_output_dir, nonaccess_output_file)

    #Storing the chromosome data in a file for accessible areas
    with open(access_output_path, 'w') as file:
        file.write("Data\n")
        for item in access_chrom_data:
            file.write(str(item) + "\n")
    #Storing the chromosome data in a file for inaccessible areas      
    with open(nonaccess_output_path, 'w') as file:
        file.write("Data\n")
        for item in nonaccess_chrom_data:
            file.write(str(item) + "\n")
            
        

ENCFF825KFE_in.txt
Average: 2905 Average from both access and non access region is too high 

After accounting the average for only accessible element, the average is: 196

ENCFF768XXW_in.txt
Average: 2378 Average from both access and non access region is too high 

After accounting the average for only accessible element, the average is: 194

ENCFF986UZO_in.txt
Average: 2762 Average from both access and non access region is too high 

After accounting the average for only accessible element, the average is: 190

ENCFF968PCT_in.txt
Average: 205 Average from both access and non access region is too high 

After accounting the average for only accessible element, the average is: 191

ENCFF928NLF_in.txt
Average: 2871 Average from both access and non access region is too high 

After accounting the average for only accessible element, the average is: 225

ENCFF615ZGF_in.txt
Average: 305 Average from both access and non access region is too high 

After accounting the average for only acces